In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
import numpy as np
import cv2
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mtcnn import MTCNN
import mediapipe as mp
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

class StopOnValLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        val_loss = logs.get('val_loss')
        if val_loss is not None and val_loss < 0.7:
            print(f"\nEpoch {epoch + 1}: Validation loss below 0.7 ({val_loss:.4f}), stopping training.")
            self.model.stop_training = True

# Initialize face detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Initialize MTCNN for registration
mtcnn_detector = MTCNN()

# FaceNet model
def load_facenet():
    # Load pre-trained FaceNet model (InceptionResNetV2)
    base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(160, 160, 3),
        pooling='avg'
    )
    
    # Freeze all layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Create embedding model
    input_layer = Input(shape=(160, 160, 3))
    x = Lambda(preprocess_input)(input_layer)
    x = base_model(x)
    embeddings = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    
    model = Model(inputs=input_layer, outputs=embeddings)
    return model

def check_nim_exists(nim, dataset_dir):
    """Check if NIM folder already exists"""
    nim_folder = os.path.join(dataset_dir, nim)
    return os.path.exists(nim_folder)

def extract_face_embeddings(image, facenet_model):
    """Extract face embeddings using FaceNet"""
    # Resize and preprocess image
    image = cv2.resize(image, (160, 160))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    # Get embedding
    embedding = facenet_model.predict(image, verbose=0)
    return embedding[0]  # Return 1D embedding vector

def load_dataset(dataset_dir, facenet_model):
    """Load dataset with FaceNet embeddings from all users"""
    X, y = [], []
    print(f"\nLoading dataset from {dataset_dir}")
    
    # Get all NIM folders
    nim_folders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]
    
    if not nim_folders:
        print("No registered users found in dataset!")
        return np.array(X), np.array(y)
    
    for nim in nim_folders:
        nim_path = os.path.join(dataset_dir, nim)
        image_path = os.path.join(nim_path, "image")
        
        if not os.path.exists(image_path):
            continue
            
        print(f"Processing NIM: {nim}")
        image_files = [f for f in os.listdir(image_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img_name in image_files:
            img_path = os.path.join(image_path, img_name)
            image = cv2.imread(img_path)
            
            if image is None:
                continue
                
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            embedding = extract_face_embeddings(image_rgb, facenet_model)
            
            if embedding is not None:
                X.append(embedding)
                y.append(nim)
    
    print(f"\nDataset Summary:")
    print(f"Total samples loaded: {len(X)}")
    print(f"Total unique NIMs: {len(set(y))}")
    
    return np.array(X), np.array(y)

def register_new_user(dataset_dir, facenet_model):
    """Register new user with automatic capture for all angles (5 images each) with user prompt between angles"""
    while True:
        nim = input("\nEnter NIM to register (or 'q' to quit): ").strip()
        if nim.lower() == 'q':
            return False, None, None
            
        if not nim.isdigit():
            print("NIM must contain only numbers!")
            continue
            
        if check_nim_exists(nim, dataset_dir):
            print(f"\nNIM {nim} already exists!")
            print("1. Register different NIM")
            print("2. Add more images to existing NIM")
            choice = input("Choose option (1/2): ")
            
            if choice == '1':
                continue
            elif choice == '2':
                print(f"\nAdding more images to NIM {nim}")
            else:
                print("Invalid choice!")
                continue
        else:
            print(f"\nRegistering new NIM: {nim}")
            
        # Create folder structure
        nim_path = os.path.join(dataset_dir, nim)
        image_path = os.path.join(nim_path, "image")
        os.makedirs(image_path, exist_ok=True)
        
        # Capture images for different angles (5 images each)
        angles = [
            ("front", "Please face the camera directly (5 images will be captured automatically) - Press SPACE when ready"),
            ("up", "Please look upwards (5 images will be captured automatically) - Press SPACE when ready"),
            ("down", "Please look downwards (5 images will be captured automatically) - Press SPACE when ready"),
            ("left", "Please turn your head to the LEFT (5 images will be captured automatically) - Press SPACE when ready"),
            ("right", "Please turn your head to the RIGHT (5 images will be captured automatically) - Press SPACE when ready")
        ]
        
        cap = cv2.VideoCapture(0)
        captured_images = 0
        
        for angle_name, instruction in angles:
            print(f"\n{instruction}")
            
            # Wait for user to be ready (SPACE key)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Display instruction
                cv2.putText(frame, instruction, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, "Press SPACE to start capture, ESC to skip", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                cv2.imshow('Registration', frame)
                
                key = cv2.waitKey(1)
                if key == 27:  # ESC to skip this angle
                    print(f"Skipping {angle_name} angle")
                    break
                elif key == 32:  # SPACE to start capturing
                    print(f"Capturing 5 {angle_name} images automatically...")
                    
                    # Automatic capture for this angle (5 images)
                    for i in range(5):
                        time.sleep(0.5)  # Reduced wait time between captures
                        
                        ret, frame = cap.read()
                        if not ret:
                            break
                            
                        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        results = mtcnn_detector.detect_faces(frame_rgb)
                        
                        if len(results) > 0:
                            x, y, w, h = results[0]['box']
                            # Expand bounding box
                            x = max(0, x - int(w * 0.2))
                            y = max(0, y - int(h * 0.2))
                            w = min(frame.shape[1] - x, int(w * 1.4))
                            h = min(frame.shape[0] - y, int(h * 1.4))
                            
                            face = frame_rgb[y:y+h, x:x+w]
                            face_resized = cv2.resize(face, (160, 160))
                            
                            # Save image with timestamp to ensure unique filenames
                            timestamp = int(time.time() * 1000)
                            img_path = os.path.join(image_path, f"{nim}_{angle_name}_{timestamp}_{i}.jpg")
                            cv2.imwrite(img_path, cv2.cvtColor(face_resized, cv2.COLOR_RGB2BGR))
                            captured_images += 1
                            
                            # Show countdown and feedback
                            cv2.putText(frame, f"Captured {i+1}/5", (x, y-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            cv2.imshow('Registration', frame)
                            cv2.waitKey(300)  # Reduced display time
                        else:
                            print(f"No face detected for {angle_name} image {i+1}")
                            continue  # Skip this capture but continue trying
                    
                    print(f"Finished capturing {angle_name} angle")
                    break
        
        cap.release()
        cv2.destroyAllWindows()
        
        print(f"\nRegistration completed for NIM: {nim}")
        print(f"Captured {captured_images} images in total")
        
        # Verify we captured exactly 25 images (5 angles × 5 images)
        if captured_images < 25:
            print(f"Warning: Only captured {captured_images} images (expected 25)")
        
        # Train model after registration
        print("\nStarting automatic model training with updated dataset...")
        model, label_encoder = train_model(dataset_dir, facenet_model)
        
        return True, model, label_encoder

def build_classifier_model(input_shape, num_classes):
    """Build the classifier model on top of FaceNet embeddings"""
    model = Sequential([
        Dense(512, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=l2(0.005)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

def train_model(dataset_dir, facenet_model):
    """Train the classifier model with all available data"""
    X, y = load_dataset(dataset_dir, facenet_model)
    
    if len(X) == 0:
        print("\nNo training data available!")
        return None, None
        
    # Encode labels (NIMs)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    
    # Split dataset (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)
    
    # Calculate class weights to handle imbalanced data
    class_weights = class_weight.compute_class_weight(
        'balanced', classes=np.unique(y_encoded), y=y_encoded)
    class_weights = dict(enumerate(class_weights))
    
    # Build classifier model
    model = build_classifier_model((X.shape[1],), len(label_encoder.classes_))
    
    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10),
        ModelCheckpoint('best_facenet_model.keras', monitor='val_accuracy', save_best_only=True),
        StopOnValLoss()
    ]
    
    # Train model
    print("\nTraining classifier model with FaceNet embeddings...")
    history = model.fit(
        X_train, y_train,
        epochs=500,
        batch_size=32,
        validation_data=(X_val, y_val),
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluate
    loss, acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"\nTraining completed. Validation accuracy: {acc*100:.2f}%")
    
    # Save label encoder
    np.save('facenet_label_encoder.npy', label_encoder.classes_)
    
    return model, label_encoder

def realtime_recognition(model, label_encoder, facenet_model):
    """Real-time recognition for all registered users using FaceNet"""
    if model is None or label_encoder is None:
        print("Model not trained yet! Please train first.")
        return
        
    cap = cv2.VideoCapture(0)
    label_map = {i:name for i,name in enumerate(label_encoder.classes_)}
    
    # Load face detection model
    face_detector = mp.solutions.face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = face_detector.process(frame_rgb)
        
        if results.detections:
            for detection in results.detections:
                # Get bounding box
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                             int(bboxC.width * iw), int(bboxC.height * ih)
                
                # Extract face ROI
                face_roi = frame_rgb[y:y+h, x:x+w]
                
                try:
                    # Resize face for FaceNet
                    face_resized = cv2.resize(face_roi, (160, 160))
                    
                    # Get FaceNet embedding
                    embedding = extract_face_embeddings(face_resized, facenet_model)
                    
                    if embedding is not None:
                        # Predict using classifier
                        predictions = model.predict(embedding.reshape(1, -1), verbose=0)
                        idx = np.argmax(predictions)
                        confidence = np.max(predictions)
                        
                        # Get NIM label
                        nim = label_map[idx]
                        
                        # Draw results
                        color = (0, 255, 0) if confidence > 0.7 else (0, 255, 255) if confidence > 0.5 else (0, 0, 255)
                        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                        cv2.putText(frame, f"NIM: {nim} ({confidence*100:.1f}%)", 
                                   (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                except Exception as e:
                    print(f"Error processing face: {e}")
                    continue
        
        cv2.imshow('Face Recognition - All Registered Users', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

def main():
    dataset_dir = 'dataset'  # Folder structure: /dataset/NIM/image/
    os.makedirs(dataset_dir, exist_ok=True)
    
    # Initialize FaceNet model
    facenet_model = load_facenet()
    
    # Initialize classifier model and label encoder
    model = None
    label_encoder = None
    
    # Try to load existing model and label encoder
    try:
        model = load_model('best_facenet_model.keras')
        label_encoder = LabelEncoder()
        label_encoder.classes_ = np.load('facenet_label_encoder.npy', allow_pickle=True)
        print("\nLoaded existing trained model with", len(label_encoder.classes_), "registered users")
    except:
        print("\nNo existing model found or error loading model")
        
        # If dataset exists but no model, train new model
        if len(os.listdir(dataset_dir)) > 0:
            print("Found existing dataset, training new model...")
            model, label_encoder = train_model(dataset_dir, facenet_model)
    
    while True:
        print("\n==== Face Recognition System with FaceNet ====")
        print("1. Register new user")
        print("2. Train/retrain model with all data")
        print("3. Real-time recognition (all users)")
        print("4. Exit")
        
        choice = input("Select option: ")
        
        if choice == '1':
            success, new_model, new_label_encoder = register_new_user(dataset_dir, facenet_model)
            if success:
                model = new_model
                label_encoder = new_label_encoder
        elif choice == '2':
            model, label_encoder = train_model(dataset_dir, facenet_model)
        elif choice == '3':
            realtime_recognition(model, label_encoder, facenet_model)
        elif choice == '4':
            print("Exiting program...")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 21s 0us/step

No existing model found or error loading model
Found existing dataset, training new model...

Loading dataset from dataset
Processing NIM: 2110511131
Processing NIM: 2110511150

Dataset Summary:
Total samples loaded: 50
Total unique NIMs: 2


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_203         │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_204         │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 921,858 (3.52 MB)

 Trainable params: 920,322 (3.51 MB)

 Non-trainable params: 1,536 (6.00 KB)


Training classifier model with FaceNet embeddings...
Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - accuracy: 0.5938 - loss: 10.6749 - val_accuracy: 0.9000 - val_loss: 9.7417 - learning_rate: 0.0010
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 1.0000 - loss: 9.0983 - val_accuracy: 1.0000 - val_loss: 9.3550 - learning_rate: 0.0010
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 8.7016 - val_accuracy: 1.0000 - val_loss: 8.9545 - learning_rate: 0.0010
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 8.3155 - val_accuracy: 1.0000 - val_loss: 8.5536 - learning_rate: 0.0010
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 7.9355 - val_accuracy: 1.0000 - val_loss: 8.1672 - learning_rate: 0.0010
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 7.5574 - val_accuracy: 1.0000 - val_loss: 7.7989 - learning_rate: 0.0010
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms

KeyboardInterrupt: 

: 